In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from PIL import Image
import caffe
import setproctitle
import os
import surgery, score

In [7]:
setproctitle.setproctitle(os.path.basename(os.getcwd()))

caffe_root = '/home/cv/hdl/caffe'
models = '{}/models'.format(caffe_root)
part = 'head'
weights = '{}/pascalpart-fcn32s/person/vgg16fc.caffemodel'.format(models)

# init
caffe.set_device(0)
caffe.set_mode_gpu()


net = caffe.Net('model/train.prototxt', caffe.TRAIN)
data_layer = net.blobs['data']
conv1_1 = net.blobs['conv1_1']
print "Data size: ", data_layer.data.shape
print "Conv1_1 layer size: ", conv1_1.data.shape

solver = caffe.SGDSolver('model/solver.prototxt'.format(models, part))
solver.net.copy_from(weights)

# surgeries
# interp_layers = [k for k in solver.net.params.keys() if 'up' in k]
# surgery.interp(solver.net, interp_layers)

net.params['score_fr'][0].data
#biases = net.params['conv1_1'][1].data
#print "Conv1_1 biases: ", biases

Data size:  (1, 3, 377, 500)
Conv1_1 layer size:  (1, 64, 575, 698)


array([[[[ 0.]],

        [[ 0.]],

        [[ 0.]],

        ..., 
        [[ 0.]],

        [[ 0.]],

        [[ 0.]]],


       [[[ 0.]],

        [[ 0.]],

        [[ 0.]],

        ..., 
        [[ 0.]],

        [[ 0.]],

        [[ 0.]]],


       [[[ 0.]],

        [[ 0.]],

        [[ 0.]],

        ..., 
        [[ 0.]],

        [[ 0.]],

        [[ 0.]]],


       ..., 
       [[[ 0.]],

        [[ 0.]],

        [[ 0.]],

        ..., 
        [[ 0.]],

        [[ 0.]],

        [[ 0.]]],


       [[[ 0.]],

        [[ 0.]],

        [[ 0.]],

        ..., 
        [[ 0.]],

        [[ 0.]],

        [[ 0.]]],


       [[[ 0.]],

        [[ 0.]],

        [[ 0.]],

        ..., 
        [[ 0.]],

        [[ 0.]],

        [[ 0.]]]], dtype=float32)

In [5]:
# parameters
[(k, v[0].data.shape) for k, v in net.params.items()]

[('conv1_1', (64, 3, 3, 3)),
 ('conv1_2', (64, 64, 3, 3)),
 ('conv2_1', (128, 64, 3, 3)),
 ('conv2_2', (128, 128, 3, 3)),
 ('conv3_1', (256, 128, 3, 3)),
 ('conv3_2', (256, 256, 3, 3)),
 ('conv3_3', (256, 256, 3, 3)),
 ('conv4_1', (512, 256, 3, 3)),
 ('conv4_2', (512, 512, 3, 3)),
 ('conv4_3', (512, 512, 3, 3)),
 ('conv5_1', (512, 512, 3, 3)),
 ('conv5_2', (512, 512, 3, 3)),
 ('conv5_3', (512, 512, 3, 3)),
 ('fc6', (4096, 512, 7, 7)),
 ('fc7', (4096, 4096, 1, 1)),
 ('score_fr', (7, 4096, 1, 1)),
 ('upscore', (7, 7, 64, 64))]

In [6]:
# deconv layer weights
print net.params['upscore'][0].data

[[[[ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   ..., 
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]]

  [[ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   ..., 
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]]

  [[ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   ..., 
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]]

  ..., 
  [[ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   ..., 
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]]

  [[ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   ..., 
   [ 0.  0.  0. ..., 

In [17]:
solver.step(100)

{'loss': array(366804.0625, dtype=float32)}

In [18]:
weights_post = net.params['conv1_1'][0].data

In [19]:
# solver.solve()  # this will run until max_iter (as defined in solver.prototxt)

In [20]:
np.array_equal(weights_pre, weights_post)

True

In [14]:
grad = net.blobs['conv1_1'].diff

In [15]:
print np.unique(grad)

[ 0.]


In [24]:
# deconv layer weights
print net.params['upscore'][0].data

[[[[ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   ..., 
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]]

  [[ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   ..., 
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]]

  [[ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   ..., 
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]]

  ..., 
  [[ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   ..., 
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]]

  [[ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   ..., 
   [ 0.  0.  0. ..., 

In [25]:
# data blob
print net.blobs['data'].data

[[[[ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   ..., 
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]]

  [[ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   ..., 
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]]

  [[ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   ..., 
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]]]]


In [26]:
# conv1_1 blob
print net.blobs['conv1_1'].data

[[[[ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   ..., 
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]]

  [[ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   ..., 
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]]

  [[ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   ..., 
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]]

  ..., 
  [[ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   ..., 
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]]

  [[ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]
   ..., 
   [ 0.  0.  0. ..., 